In [2]:
import pandas as pd
import numpy as np
import duckdb
import matplotlib.pyplot as plt

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [16]:
DATA_PATH = "data"
sales = pd.read_csv(f"{DATA_PATH}/sales_train_validation.csv")
calendar = pd.read_csv(f"{DATA_PATH}/calendar.csv")
prices = pd.read_csv(f"{DATA_PATH}/sell_prices.csv")

print(sales.shape, calendar.shape, prices.shape)

(30490, 1919) (1969, 14) (6841121, 4)


In [17]:
sales.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,d_35,d_36,d_37,d_38,d_39,d_40,d_41,d_42,d_43,d_44,...,d_1864,d_1865,d_1866,d_1867,d_1868,d_1869,d_1870,d_1871,d_1872,d_1873,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,1,3,0,0,0,1,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,2,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,2,0,1,0,...,5,3,1,0,0,0,1,2,3,0,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,2,1,1,0,3,1,1,2,1,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [19]:
sales_ca = sales[sales["state_id"] == "CA"]
sales_ca.shape

(12196, 1919)

In [21]:
sales_ca = sales_ca[sales_ca["store_id"].isin(["CA_1", "CA_2"])]
sales_ca["store_id"].value_counts()

store_id
CA_1    3049
CA_2    3049
Name: count, dtype: int64

In [22]:
sales_ca = sales_ca[sales_ca["cat_id"] == "FOODS"]
sales_ca.shape

(2874, 1919)

In [25]:
# Select only columns that start with 'd_' for value_vars
value_vars = [col for col in sales_ca.columns if col.startswith("d_")]

sales_long = sales_ca.melt(
    id_vars=["item_id", "dept_id", "cat_id", "store_id", "state_id"],
    value_vars=value_vars,
    var_name="d",
    value_name="units_sold"
)

top_items = (
    sales_long.groupby("item_id")["units_sold"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

top_items

Index(['FOODS_3_090', 'FOODS_3_586', 'FOODS_3_252', 'FOODS_3_714',
       'FOODS_3_120'],
      dtype='object', name='item_id')

In [26]:
sales_subset = sales_long[sales_long["item_id"].isin(top_items)]
sales_subset.shape

(19130, 7)

In [28]:
calendar_subset = calendar[["d", "date", "wm_yr_wk", "event_name_1", "snap_CA"]]

df = sales_subset.merge(calendar_subset, on="d", how="left")
df["date"] = pd.to_datetime(df["date"])

df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,units_sold,date,wm_yr_wk,event_name_1,snap_CA
0,FOODS_3_090,FOODS_3,FOODS,CA_1,CA,d_1,107,2011-01-29,11101,NaN,0
1,FOODS_3_120,FOODS_3,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,NaN,0
2,FOODS_3_252,FOODS_3,FOODS,CA_1,CA,d_1,19,2011-01-29,11101,NaN,0
3,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_1,42,2011-01-29,11101,NaN,0
4,FOODS_3_714,FOODS_3,FOODS,CA_1,CA,d_1,49,2011-01-29,11101,NaN,0


In [29]:
df = df.merge(
    prices,
    on=["store_id", "item_id", "wm_yr_wk"],
    how="left"
)

df.head()

,item_id,dept_id,cat_id,store_id,state_id,d,units_sold,date,wm_yr_wk,event_name_1,snap_CA,sell_price
0,FOODS_3_090,FOODS_3,FOODS,CA_1,CA,d_1,107,2011-01-29,11101,NaN,0,1.25
1,FOODS_3_120,FOODS_3,FOODS,CA_1,CA,d_1,0,2011-01-29,11101,NaN,0,NaN
2,FOODS_3_252,FOODS_3,FOODS,CA_1,CA,d_1,19,2011-01-29,11101,NaN,0,1.48
3,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_1,42,2011-01-29,11101,NaN,0,1.48
4,FOODS_3_714,FOODS_3,FOODS,CA_1,CA,d_1,49,2011-01-29,11101,NaN,0,1.48


In [30]:
df.isnull().mean().sort_values(ascending=False).head(10)

event_name_1    0.919498
sell_price      0.038055
item_id         0.000000
dept_id         0.000000
cat_id          0.000000
store_id        0.000000
state_id        0.000000
d               0.000000
units_sold      0.000000
date            0.000000
dtype: float64

In [31]:
df.to_parquet("data/clean_base_table.parquet", index=False)